# DM - Système de décision

## PARTIE 1 : Gurobi

In [1]:
import numpy as np
from gurobipy import Model
from gurobipy.gurobipy import GRB
import random

In [2]:
def generate_datasets(students_count, subject_counts):
    students_a = random.randint(students_count//4, students_count-(students_count//4))
    students_r = students_count - students_a
    
    A = np.random.randint(6, 21, (students_a, subject_counts))
    R = np.random.randint(0, 15, (students_r, subject_counts))
    
    S = np.concatenate((A, R))
    
    return A, R, S
    #return np.concatenate((A, R)), students_a, students_r

In [3]:
A, R, S = generate_datasets(10, 5) #10 étudiants et 5 matières
print("Elèves ayant réussi :\n", A)
print("\nElèves ayant échoué :\n", R)

Elèves ayant réussi :
 [[ 9 16 19 18 13]
 [14 11 11 10 18]
 [20 18 18 14 15]
 [12  8 18 17 20]
 [ 7 12 11  9 13]
 [19 20 20  7  7]
 [14 15 16  9 16]]

Elèves ayant échoué :
 [[ 3  4 13  3 12]
 [ 5 12  2 11  9]
 [ 7 13  4  9 13]]


In [4]:
def solve(student_count, subject_count):

    A, R, S = generate_datasets(student_count, subject_count)
    print("Elèves ayant réussi :\n", A)
    print("\nElèves ayant échoué :\n", R)

    # Instanciation du modèle
    m = Model("PL modeling")
    
    #------------------VARIABLES------------------

    # Vecteur des coefficients des différentes matières
    w = m.addMVar(shape=subject_count, lb=0, ub=1, name="w")

    lambdou = m.addVar(name="lambda", lb=0.5, ub=1)

    #Pour les élèves qui valident (correspond à )
    sigma1 = m.addMVar(shape=len(A), name="x")
    #Pour 
    sigma2 = m.addMVar(shape=len(R), name="y")
    
    alphou = m.addVar(name="alpha")
    
    #Mise à jour du modèle
    m.update()
    
    
    #------------------CONTRAINTES------------------
    
    #Somme des coeffs
    m.addConstr(w.sum() == 1)
    m.update()
    
    #Contraintes des acceptés
    for index, line in enumerate(A):
        marks = [weight for passed, weight in zip(line>=12, w) if passed]
        if (len(marks)>0):
            m.addConstr(sum(marks)  - sigma1[index] == lambdou)
    
    #Contraintes des refusés
    for index, line in enumerate(R):
        marks = [weight for passed, weight in zip(line>=12, w) if passed]
        if (len(marks)>0):
            m.addConstr(sum(marks) == lambdou - sigma2[index])
    
    for student_offset in sigma1: m.addConstr(alphou<=student_offset)
        
    for student_offset in sigma2: m.addConstr(alphou<=student_offset)
        
    m.update()
        

    #Fonction objectif
    m.setObjective(alphou, GRB.MAXIMIZE)
    m.update()

    
    # Affichage en mode texte du PL
    print("__________________________________________________")
    print("\nContraintes de notre modèle :\n")
    print(m.display())
    print("__________________________________________________\n")

    
    #------------------RESOLUTION------------------
    m.optimize()
    
    #Affichage de notre solution optimale
    print("\nLes coefficients de notre solution optimale sont :")
    print(f"__________________")
    
    print(f"| Matière  |coeff|")

    for index, coef in enumerate(w.X) : 
        print(f"|matière {index+1} | {coef} |")
    print(f"__________________")

    print(f"alpha = {alphou.X} \nlambda = {lambdou.X}")


In [6]:
solve(10, 5)

Elèves ayant réussi :
 [[11 13 11 12 17]
 [ 6  7 20  6 11]
 [ 9 17 13 11  6]
 [15 12 18  9 17]
 [ 7 10 12 17 10]
 [16 10 12  8 10]
 [17 19  6 13 16]
 [12 10 19 16 14]]

Elèves ayant échoué :
 [[14  9 13  0 14]
 [13  9  4 13 10]]
__________________________________________________

Contraintes de notre modèle :

Maximize
  <gurobi.LinExpr: alpha>
Subject To
  R0: <gurobi.LinExpr: w[0] + w[1] + w[2] + w[3] + w[4]> = 1
  R1: <gurobi.LinExpr: w[1] + w[3] + w[4] + -1.0 lambda + -1.0 x[0]> = 0
  R2: <gurobi.LinExpr: w[2] + -1.0 lambda + -1.0 x[1]> = 0
  R3: <gurobi.LinExpr: w[1] + w[2] + -1.0 lambda + -1.0 x[2]> = 0
  R4: <gurobi.LinExpr: w[0] + w[1] + w[2] + w[4] + -1.0 lambda + -1.0 x[3]> = 0
  R5: <gurobi.LinExpr: w[2] + w[3] + -1.0 lambda + -1.0 x[4]> = 0
  R6: <gurobi.LinExpr: w[0] + w[2] + -1.0 lambda + -1.0 x[5]> = 0
  R7: <gurobi.LinExpr: w[0] + w[1] + w[3] + w[4] + -1.0 lambda + -1.0 x[6]> = 0
  R8: <gurobi.LinExpr: w[0] + w[2] + w[3] + w[4] + -1.0 lambda + -1.0 x[7]> = 0
  R9: <guro